In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen # changed python 2 code: from urllib import urlopen
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import re
import scipy.stats
import numpy as np


%matplotlib inline

## Getting game reviews (ordered from best to worst, 158 pages total)

In [2]:
User_Agent = "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.96 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"

In [3]:
url = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?sort=desc&page=0'

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 
request=urllib.request.Request(url,None,headers) 
response = urllib.request.urlopen(request)
data = response.read()

In [5]:
soup = BeautifulSoup(data, 'lxml')

In [6]:
import time

In [7]:

df = pd.DataFrame(columns=["title", "platform","critic_score", "user_score", "release_date"])

for page in range(157):
    url = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?sort=desc&page=' + str(page)

    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    request=urllib.request.Request(url,None,headers) 
    response = urllib.request.urlopen(request)
    data = response.read()
    soup = BeautifulSoup(data, 'lxml')
    print(page)
    for games in soup.find_all('div', {'class':['product_row game', 'product_row game first']}):
        title_and_platform = games.find('div', class_='product_item product_title').find('a').text    
        title_and_platform = str(title_and_platform).replace("  ", "") # removes any spaces more than one in a row
        title = re.findall('(.*?)\n', title_and_platform)[1] # takes the second line, which is the title
        platform = (re.findall('\n\((.*?)\)', title_and_platform))[0]
        critic_score = games.find('div', class_='product_item product_score').text
        critic_score = str(critic_score).replace("  ", "") # removes any spaces more than one in a row
        critic_score = str(critic_score).replace('\n', "") # removes any new lines
        user_score = games.find('span', attrs = {"class": re.compile("(data textscore .*?)")}).text     
        # user_score = games.find('div', attrs = {"class": "product_item product_userscore_txt"}).find('span', attrs = {"class": re.compile("(data textscore .*?)")}).contents # this works equally well, and is a more careful coding in case "data textscore" was reused somewhere else
        if user_score != 'tbd':
            user_score = int(float(str(user_score))*10) # The critic scores are out of 100, while the user scores are out of 10 - normalize them here
        release_date = games.find('div', class_='product_item product_date').text
        release_date = str(release_date).replace("  ", "") # removes any spaces more than one in a row
        release_date = str(release_date).replace('\n', "") # removes any new lines
        df = df.append({"title":title, "platform":platform, "critic_score": critic_score, "user_score": user_score, "release_date": release_date}, ignore_index=True) 
    time.sleep(3)
df.head() 
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156


,title,platform,critic_score,user_score,release_date
0,The Legend of Zelda: Ocarina of Time,N64,99,91,"Nov 23, 1998"
1,Tony Hawk's Pro Skater 2,PS,98,74,"Sep 20, 2000"
2,Grand Theft Auto IV,PS3,98,75,"Apr 29, 2008"
3,SoulCalibur,DC,98,87,"Sep 8, 1999"
4,Grand Theft Auto IV,X360,98,79,"Apr 29, 2008"


In [8]:
df.head()

,title,platform,critic_score,user_score,release_date
0,The Legend of Zelda: Ocarina of Time,N64,99,91,"Nov 23, 1998"
1,Tony Hawk's Pro Skater 2,PS,98,74,"Sep 20, 2000"
2,Grand Theft Auto IV,PS3,98,75,"Apr 29, 2008"
3,SoulCalibur,DC,98,87,"Sep 8, 1999"
4,Grand Theft Auto IV,X360,98,79,"Apr 29, 2008"


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15543 entries, 0 to 15542
Data columns (total 5 columns):
title           15543 non-null object
platform        15543 non-null object
critic_score    15543 non-null object
user_score      15543 non-null object
release_date    15543 non-null object
dtypes: object(5)
memory usage: 607.2+ KB


## So here we have 15,543... all of the games from all platforms in metacritic's database